In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import time
import os

# Load Data

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
data_local = pd.read_excel("../../analisa/2025/Summary_2025.xlsx")

In [4]:
data_local['Source'] = "Web Ads"
data_local.to_excel("../../analisa/2025/Summary_2025.xlsx")

Load data from sql

In [5]:
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# Database credentials
username = "wp_plegw"
password = "VD@L9l9r_3%O4wzF"
host = "103.229.73.62"
port = 3306
database = "wp_ajskr"

# URL-encode username & password in case they have special characters
username_enc = quote_plus(username)
password_enc = quote_plus(password)

# Create SQLAlchemy engine
engine = create_engine(
    f"mysql+pymysql://{username_enc}:{password_enc}@{host}:{port}/{database}"
)

# Run a query and load results into Pandas
query_donasi = """SELECT a.*,b.title
FROM wp_ajskr.tEZ3UbOt_dja_donate a
LEFT JOIN wp_ajskr.tEZ3UbOt_dja_campaign b ON b.campaign_id = a.campaign_id """
query_aff_submit = """SELECT *
FROM wp_ajskr.tEZ3UbOt_dja_aff_submit"""
query_aff_code = """SELECT *
FROM wp_ajskr.tEZ3UbOt_dja_aff_code"""
query_users = """SELECT *
FROM wp_ajskr.tEZ3UbOt_users"""

donasi_table = pd.read_sql(query_donasi, engine)
aff_submit_table = pd.read_sql(query_aff_submit, engine)
aff_code_table = pd.read_sql(query_aff_code, engine)
user_table = pd.read_sql(query_users, engine)

In [6]:
# Join Table
donasi_submit = donasi_table.merge(aff_submit_table, left_on='id', right_on='donate_id', how='left')
donasi_submit_code = donasi_submit.merge(aff_code_table, left_on='affcode_id', right_on='id', how='left')
donasi_users = donasi_submit_code.merge(user_table, left_on='user_id_y', right_on='ID', how='left')

In [7]:
# Choose only necessary columns
donasi_download = donasi_users[['campaign_id','invoice_id', 'name', 'whatsapp', 'email', 'ip', 'comment', 'title','nominal', 'payment_method', 'payment_number', 'payment_account', 'status', 'nominal_commission', 'display_name', 'aff_code', 'created_at_x', 'info_qurban', 'info_package2', 'info_zfitrah', 'info_donate', 'utm_source', 'utm_medium', 'utm_content', 'utm_campaign', 'utm_term', 'utm_id']]

# Mapping campaign and campaigner
campaign_map = {
    "dja3d0s0b0k":"pungky",
    "djah4o94i2w":"cekat",
    "djazb6xk1xx": "fahry",
    "djayr4jyoxw": "fahry",
    "djax0gtszee": "alif",
    "djauvpug3dk": "fahry",
    "djar8al0qg2": "ali",
    "djam1ptsz9a": "dedi",
    "djait62m3qv": "fatih",
    "djah4o94i2w": "cekat",
    "djabkim8p3t": "fredo",
    "dja9o7ftxxf": "cekat"
}

donasi_download["zisco"] = donasi_download["campaign_id"].map(campaign_map)  # returns "alif" or None if not found
donasi_download['zisco'].value_counts(dropna=False)

C:\Users\faruq\AppData\Local\Temp\ipykernel_7308\2603132602.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  donasi_download["zisco"] = donasi_download["campaign_id"].map(campaign_map)  # returns "alif" or None if not found


zisco
fredo     19977
fahry     18398
alif      13494
fatih     11933
NaN        2693
ali         955
dedi        319
cekat        15
pungky        4
Name: count, dtype: int64

# Feature engineering data from SQL

In [8]:
donasi_download['Day'] = donasi_download['created_at_x'].dt.strftime("%a")
donasi_download['Time'] = donasi_download['created_at_x'].dt.strftime("%H:%M:%S")
donasi_download = donasi_download.rename(columns={
    "invoice_id": "Invoice ID",
    "name": "Donatur",
    "whatsapp": "Whatsapp",
    "email": "Email",
    "comment": "Comment",
    "title": "Program",
    "nominal":"Total",
    "payment_method": "Payment Method",
    "payment_number": "Payment Number",
    "payment_account": "Payment Account",
    "status": "Payment Status",
    "nominal_commission":"Fundraiser Commision",
    "display_name":"Fundraiser Name",
    "created_at_x":"Date",
    "info_qurban":"Data Qurban",
    "info_package2":"Data Package-2",
    "info_zfitrah":"Data Zakat Fitrah",
    "info_donate":"Additional Data",
    "utm_source":"UTM Source",
    "utm_medium":"UTM Medium",
    "utm_content":"UTM Content",
    "utm_campaign":"UTM Campaign",
    "utm_term":"UTM Term",
    "utm_id":"UTM ID"    
})

donasi_download["Payment Status"] = np.where(donasi_download['Payment Status'] == 1, "Success", "Waiting")

donasi_download['Source'] = "Web Ads"

C:\Users\faruq\AppData\Local\Temp\ipykernel_7308\228523208.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  donasi_download['Day'] = donasi_download['created_at_x'].dt.strftime("%a")
C:\Users\faruq\AppData\Local\Temp\ipykernel_7308\228523208.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  donasi_download['Time'] = donasi_download['created_at_x'].dt.strftime("%H:%M:%S")


In [ ]:
# save data donation from database to local
donasi_download.to_excel("data_payment.xlsx")

# 
data_from_web = pd.read_excel("Data_baru.xlsx")


data_from_web.to_excel("Data_baru.xlsx")

Load data sql that frequently updated by donasi download

In [11]:
def update_dataframe(main_df: pd.DataFrame, updated_df: pd.DataFrame, cols_to_check: list) -> pd.DataFrame:
    """
    Update main_df with changes from updated_df based on specified columns.
    - Rows with different values in cols_to_check will be updated.
    - New rows not present in main_df will be added.

    Parameters:
    -----------
    main_df : pd.DataFrame
        The original DataFrame (e.g. existing data from web or database).
    updated_df : pd.DataFrame
        The new DataFrame with possibly updated or new rows.
    cols_to_check : list
        List of columns to check for changes.

    Returns:
    --------
    pd.DataFrame
        Updated DataFrame with changes applied.
    """

    # # 1. Find common indices 
    common_ids = updated_df.index.intersection(main_df.index) 
    # 2. Identify rows that have differences 
    diff_mask = (updated_df.loc[common_ids, cols_to_check].ne(main_df.loc[common_ids, cols_to_check]).any(axis=1))
    # 3. Extract rows to update 
    rows_to_update = updated_df.loc[common_ids[diff_mask]] 
    # 4. Identify new rows (not in main_df) 
    new_rows = updated_df.loc[~updated_df.index.isin(main_df.index)] 
    # 5. Apply updates and add new rows 
    main_df.update(rows_to_update) 
    updated_main_df = pd.concat([main_df, new_rows]) 
    # Optional: sort index if needed 
    updated_main_df = updated_main_df.sort_index() 
    # Info 
    print("✅ Excel updated successfully:") 
    print(f"- {len(new_rows)} new rows added") 
    print(f"- {len(rows_to_update)} rows updated based on {cols_to_check}") 
    return updated_main_df




def update_data_web(main_df: pd.DataFrame, updated_df: pd.DataFrame, key_col: str, cols_to_check: list) -> pd.DataFrame:
    import pandas as pd

    main_df = main_df.copy()
    updated_df = updated_df.copy()

    # --- Pastikan kolom kunci unik ---
    if main_df[key_col].duplicated().any():
        print(f"⚠️ Duplicate keys found in main_df on '{key_col}', keeping first.")
        main_df = main_df.drop_duplicates(subset=key_col, keep='first')

    if updated_df[key_col].duplicated().any():
        print(f"⚠️ Duplicate keys found in updated_df on '{key_col}', keeping first.")
        updated_df = updated_df.drop_duplicates(subset=key_col, keep='first')

    # --- Jadikan kolom kunci sebagai index ---
    main_df = main_df.set_index(key_col)
    updated_df = updated_df.set_index(key_col)

    # --- Ambil key yang sama di kedua dataframe ---
    common_ids = updated_df.index.intersection(main_df.index)

    # --- 1️⃣ Cari baris yang berbeda di kolom tertentu ---
    diff_mask = main_df.loc[common_ids, cols_to_check].ne(updated_df.loc[common_ids, cols_to_check]).any(axis=1)

    # --- 2️⃣ Ambil baris yang berbeda dari updated_df untuk overwrite ---
    rows_to_overwrite = updated_df.loc[common_ids[diff_mask]]

    # --- 3️⃣ Overwrite baris yang berbeda ---
    main_df.loc[rows_to_overwrite.index] = rows_to_overwrite

    # --- 4️⃣ Tambahkan baris baru yang belum ada ---
    new_rows = updated_df.loc[~updated_df.index.isin(main_df.index)]
    updated_main_df = pd.concat([main_df, new_rows])

    # --- 5️⃣ Reset index ke kolom kunci ---
    updated_main_df = updated_main_df.reset_index()

    # --- 6️⃣ Log hasil ---
    print("✅ Overwrite update completed:")
    print(f"- {len(rows_to_overwrite)} rows overwritten (different in {cols_to_check})")
    print(f"- {len(new_rows)} new rows added")
    print(f"- Final total rows: {len(updated_main_df)}")

    return updated_main_df


In [12]:
data_from_web_updated = update_data_web(
    main_df=data_from_web,
    updated_df=donasi_download,
    key_col="Invoice ID",
    cols_to_check=["Payment Status"]
)

⚠️ Duplicate keys found in main_df on 'Invoice ID', keeping first.
⚠️ Duplicate keys found in updated_df on 'Invoice ID', keeping first.


C:\Users\faruq\AppData\Local\Temp\ipykernel_7308\871079552.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['082295409840' '082163232273' '082324254488' '085298029119'
 '081247104445' '085333738069' '0881311964250' '0881315816284'
 '082261488406' '082386662861' '085377999881' '08129279973' '08127256610'
 '087895162548' '081809429700' '085963008090' '08111011808' '081368253770'
 '081270892318' '081361183468' '081999228230' '085268274398'
 '085796730876' '081372156814' '085700909080' '081310266642'
 '082175141569' '081246086390' '085368250439' '085247072344'
 '081362434311' '089668010707' '089522977143' '085759464210'
 '085243860169' '081334633599' '081341296912' '085742446373'
 '085379966990' '08819123657' '081586812189' '085360395395' '082319540240'
 '081244721421' '081218856817' '0895402971394' '085263676604'
 '081373604781' '085704199409' '082129971844' '083139819619'
 '089673090117' '085343522121

✅ Overwrite update completed:
- 284 rows overwritten (different in ['Payment Status'])
- 8782 new rows added
- Final total rows: 67986


In [13]:
# df_final.reset_index(drop=True, inplace=True)

# 🔥 Also remove any "Unnamed" columns just in case
# df_final = df_final.loc[:, ~df_final.columns.str.contains("^Unnamed")]
data_from_web_updated = data_from_web_updated[['Invoice ID', 'Donatur', 'Whatsapp',
       'Email', 'ip', 'Comment', 'Program', 'Total', 'Payment Method',
       'Payment Number', 'Payment Account', 'Payment Status',
       'Fundraiser Commision','aff_code', 'Fundraiser Name','zisco', 'Date', 'Day', 'Time', 'Data Qurban',
       'Data Package-2', 'Data Zakat Fitrah', 'Additional Data', 'UTM Source',
       'UTM Medium', 'UTM Content', 'UTM Campaign', 'UTM Term', 'UTM ID','Source']]

data_from_web_updated.to_excel("checkpoint_files/data_from_web_updated.xlsx")

# Next Step (main)

In [14]:
data_local['Invoice ID'].duplicated().sum()

np.int64(0)

In [15]:
data_local_clean = data_local[~data_local["Invoice ID"].isin(data_from_web["Invoice ID"])]

In [16]:
data_all = pd.concat([data_local_clean, data_from_web_updated])
data_all

,Unnamed: 0.20,Unnamed: 0.19,Unnamed: 0.18,Unnamed: 0.17,Unnamed: 0.16,Unnamed: 0.15,Unnamed: 0.14,Unnamed: 0.13,Unnamed: 0.12,Unnamed: 0.11,Unnamed: 0.10,Unnamed: 0.9,Unnamed: 0.8,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,No,Invoice ID,Donatur,Sapaan,Nominal,Kode Unik,Total,Whatsapp,Email,Comment,Program,Payment Method,Payment Number,Payment Account,Payment Status,Fundraiser Commission,Fundraiser Name,Date,Day,Time,UTM Source,UTM Medium,Source,ip,Fundraiser Commision,aff_code,zisco,Data Qurban,Data Package-2,Data Zakat Fitrah,Additional Data,UTM Content,UTM Campaign,UTM Term,UTM ID
5,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,INV-2508040IVSI,Mayang,Bapak,100000.0,446.0,100446.0,82162426297.0,NaN,NaN,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,instant,https://m.dana.id/s/m7v7pwbn,DANA,Waiting,1004.0,Teman Baik 1,2025-08-04 00:00:00,Mon,13:36:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,7.0,INV-250804R2C2K,Isep Kamiludin,Bapak,100000.0,528.0,100528.0,87772391069.0,NaN,Semoga Allah mengabulkqn doa kita semua..aamiin,BANTU PALESTINA SEKARANG,instant,https://m.dana.id/s/49unkdhk,DANA,Waiting,1005.0,Teman Baik 1,2025-08-04 00:00:00,Mon,13:36:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,54.0,55.0,INV-2508049SZY8,A.nabila,Ibu,100000.0,589.0,100589.0,812812335.0,NaN,NaN,SEDEKAH PALESTINA,instant,https://flip.id/pwf/transaction/consolidated?r...,QRIS,Waiting,1006.0,Teman Baik 1,2025-08-04 00:00:00,Mon,10:52:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,56.0,57.0,INV-250804LQMVG,A.nabila,Ibu,100000.0,209.0,100209.0,812812335.0,NaN,NaN,SEDEKAH PALESTINA,transfer,BSI 7772526274,BSI,Waiting,1002.0,Teman Baik 1,2025-08-04 00:00:00,Mon,10:49:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,72.0,73.0,INV-250804DZ01S,Didi sutardi.didi1968@gmail.com,Bapak,50000.0,640.0,50640.0,81324233883.0,didisutardi0568@gmail.com,Semoga Allah cepat akhiri penderitaan Rakyat d...,BANTU PALESTINA SEKARANG,instant,https://api.midtrans.com/v2/gopay/d1730895-e65...,GOPAY,Waiting,506.0,Teman Baik 1,2025-08-04 00:00:00,Mon,10:12:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INV-25112297ZCF,HR lbs,NaN,NaN,NaN,100932.0,08126558338,,,INFAQ SUDAN: BANTUAN KEMANUSIAAN UNTUK SUDAN,transfer,Mandiri 1410000665448,Yayasan Teman Jalan Kebaikan,Success,NaN,Teman Baik 1,2025-11-22 09:28:11,Sat,09:28:11,ig,Instagram_Reels,Web Ads,182.9.35.17,1009.0,8bmgx,NaN,[],[],[],"{""Kode Unik"":""932""}",Iklan Penjualan Baru,SUDAN 1 TB - Salin,120238002435220693,120238002435190693
67982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,INV-251122CW1PT,Muliono,NaN,NaN,NaN,100652.0,085360206990,moellalfatih844@gmail.com,Semoga derita rakyat sudan segera berakhir,INFAQ SUDAN: BANTUAN KEMANUSIAAN UNTUK SUDAN,transfer,BSI 7772526274,Yayasan Teman Jalan Kebaikan,Waiting,NaN,Teman Baik 1,2025-11-22 09:33:22,Sat,09:33:22,ig,Instagram_Feed,Web Ads,114.122.36.93,1007.0,8bmgx,NaN,[],[],[],"{""Kode Unik"":""652""}",Iklan Penjualan Baru,SUDAN TZ - Salin,120235370722780046,120235370722850046
67983,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [17]:
data_all = data_all.dropna(subset=["Invoice ID"])

In [18]:
data_all.to_excel("Data_transaksi.xlsx")

In [19]:
data_all['Whatsapp'] = pd.to_numeric(data_all['Whatsapp'], errors='coerce')
data_all = data_all.dropna(subset=['Whatsapp'])
data_all.info()

C:\Users\faruq\AppData\Local\Temp\ipykernel_7308\1463130004.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all['Whatsapp'] = pd.to_numeric(data_all['Whatsapp'], errors='coerce')


<class 'pandas.core.frame.DataFrame'>
Index: 74617 entries, 5 to 67985
Data columns (total 56 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Unnamed: 0.20          6633 non-null   float64       
 1   Unnamed: 0.19          6633 non-null   float64       
 2   Unnamed: 0.18          6633 non-null   float64       
 3   Unnamed: 0.17          6633 non-null   float64       
 4   Unnamed: 0.16          6633 non-null   float64       
 5   Unnamed: 0.15          6633 non-null   float64       
 6   Unnamed: 0.14          6633 non-null   float64       
 7   Unnamed: 0.13          6633 non-null   float64       
 8   Unnamed: 0.12          6633 non-null   float64       
 9   Unnamed: 0.11          6633 non-null   float64       
 10  Unnamed: 0.10          6633 non-null   float64       
 11  Unnamed: 0.9           6633 non-null   float64       
 12  Unnamed: 0.8           6633 non-null   float64       
 13  Unname

In [20]:
# change data type and make a new necessary columns 
data_all['Whatsapp'] = data_all['Whatsapp'].astype(int)
data_all['Time'] = pd.to_datetime(data_all['Time'], format='%H:%M:%S')  # ← keep full datetime
data_all['hour'] = data_all['Time'].dt.hour
data_all['label_jam'] = data_all['hour'].fillna(0).apply(
    lambda h: f"{int(h):02d}-{(int(h) + 1) % 24:02d}"
)

In [21]:
# Standarize whatsapp number
def is_random_number(num):
    num = str(num)

    # Nomor harus hanya angka
    if not num.isdigit():
        return False

    # Harus valid dari segi panjang dan awalan
    starts_valid = num.startswith("08") or num.startswith("8") or num.startswith("62")
    length_valid = 8 < len(num) <= 14

    if not (starts_valid and length_valid):
        return False

    # Cek jika terlalu banyak angka sama (seperti 08888888888)
    if len(set(num)) <= 3:
        return False

    # Cek pola berulang (seperti 081234567812345678)
    if len(num) > 6 and num[:int(len(num)/2)] == num[int(len(num)/2):]:
        return False

    # Cek pola urutan (seperti 08123456789)
    if num[2:].isdigit() and num[2:] in '1234567890'*2:
        return False

    return True

data_all['is_random'] = data_all['Whatsapp'].apply(is_random_number)
data_all['kategori_nomor'] = data_all['is_random'].map({True: 'Nomor Acak', False: 'Nomor Pola/Tidak Valid'})
data_all[data_all['is_random'] == False].head()

,Unnamed: 0.20,Unnamed: 0.19,Unnamed: 0.18,Unnamed: 0.17,Unnamed: 0.16,Unnamed: 0.15,Unnamed: 0.14,Unnamed: 0.13,Unnamed: 0.12,Unnamed: 0.11,Unnamed: 0.10,Unnamed: 0.9,Unnamed: 0.8,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,No,Invoice ID,Donatur,Sapaan,Nominal,Kode Unik,Total,Whatsapp,Email,Comment,Program,Payment Method,Payment Number,Payment Account,Payment Status,Fundraiser Commission,Fundraiser Name,Date,Day,Time,UTM Source,UTM Medium,Source,ip,Fundraiser Commision,aff_code,zisco,Data Qurban,Data Package-2,Data Zakat Fitrah,Additional Data,UTM Content,UTM Campaign,UTM Term,UTM ID,hour,label_jam,is_random,kategori_nomor
7218,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,7218.0,508.0,INV-250720QFDBQ,Hamba Alloh,Bapak,50000.0,782.0,50782.0,81000000,NaN,Aamiin,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,transfer,Mandiri 1410000665448,Mandiri,Waiting,508.0,Teman Baik 1,2025-07-20,Sun,1900-01-01 04:37:00,ig,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,04-05,False,Nomor Pola/Tidak Valid
27138,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,27138.0,494.0,INV-2505242MJ04,Rafli Firyal,Bapak,100000.0,547.0,103877.0,87777777777,NaN,NaN,INFAK PALESTINA,va,1900800023227110,BCA VA,Success,1039.0,Teman Baik 1,2025-05-24,Sat,1900-01-01 02:11:00,ig,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,02-03,False,Nomor Pola/Tidak Valid
27226,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,27226.0,582.0,INV-2505234S3PA,Hamba Allah,Ibu,100000.0,169.0,100169.0,123506328,NaN,Bismillah hamba donasi karena Allah Barokallah...,INFAK PALESTINA,transfer,BSI 7772526274,BSI,Waiting,1002.0,Teman Baik 1,2025-05-23,Fri,1900-01-01 13:12:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,13-14,False,Nomor Pola/Tidak Valid
27287,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,27287.0,643.0,INV-250523EQSG2,Hambaallah,Bapak,100000.0,628.0,100628.0,999086705,NaN,NaN,INFAK PALESTINA,transfer,BSI 7772526274,BSI,Waiting,1006.0,Teman Baik 1,2025-05-23,Fri,1900-01-01 05:54:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,05-06,False,Nomor Pola/Tidak Valid
27439,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,27439.0,795.0,INV-250522IEGUE,Alwi nopriansyah,Bapak,100000.0,76.0,100076.0,8131191,NaN,NaN,INFAK PALESTINA,transfer,Mandiri 1410000665448,Mandiri,Waiting,1001.0,Teman Baik 1,2025-05-22,Thu,1900-01-01 02:05:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,02-03,False,Nomor Pola/Tidak Valid


In [22]:
# Standarisasi nomor agar diawali dengan 62
def format_nomor(nomor):
    nomor = str(nomor)
    if nomor.startswith('6'):
        return nomor
    elif nomor.startswith('8'):
        return '62' + nomor
    else:
        return nomor  # Bisa disesuaikan jika ada kondisi lain
    
data_all['Whatsapp'] = data_all['Whatsapp'].apply(format_nomor)

# Rekap himpunan

In [23]:
# load processed data from web
data_with_rekapan = pd.read_excel("checkpoint_files/data_with_updated_himpunan.xlsx")

# gsheet id each CRM rekap himpunan
cs_zein = "17mDDgZm5jrbOUZU2rhFMiC999FvhJuTtvooqs5gZAaY"
cs_vicky = "1e1uyBU_1MtbdktSrnqHj-f83t7DM17S8_LFGMqYcccQ"
cs_intan = "1CUC2hdiJ2CkKDv3d5vmLdtEI4aawz7ERV7E0ChwWjxM"
cs_diah = "12ARtZE3RVK87uKPvzNbPxqouzZnEB3cAtI6WVv3emiA"
cs_shania = "1LmfIz2ZARiROFnSOpiG1oRxYI_hV0ZdWPxeW1LJT1c8"
cs_firda = "10BLcY2mY904pTu5uAeKyFs4UhI6WAyz68zcW70Odw28"
cs_endah = "1hGYcODnzpNyzYfLiGE4UU8SNx7yV12NqmOOpEVthtC8"
cs_erni = "1vvRzfvm64b5yNLzt_4FKO2XpJp25vV66s8tbr3F3InE"
cs_agil = "1lSffBtyrLEa5qkoqzVdHxYCSqprpzM6EV4sGmIp0aC0"
cs_oliv = "1X9Sv4LbRHLFv5LLzKhPxlRHWUB_vA6byxdoWUNFE1Gw"



# rekap himpunan
rekap_himpunan_wa_Juni = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_Juni.xlsx")
rekap_himpunan_wa_7778 = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_7778.xlsx")
rekap_himpunan_wa_7778_Juli = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_Juli.xlsx")
rekap_himpunan_wa_2061 = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_2061.xlsx")
rekap_himpunan_wa_2062 = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_2062.xlsx")
rekap_himpunan_wa_2060_juli = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_2060_Juli.xlsx")

# agus
rekap_himpunan_wa_2060_agustus = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_2060_Agustus.xlsx")
rekap_himpunan_0056_0036_agustus = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_Agustus zein.xlsx")

# sept
rekap_himpunan_wa_2060_september = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_2060_September.xlsx")
rekap_himpunan_2056_2057_sept = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap_Himpunan_2056-2057_Sept.xlsx")
rekap_himpunan_0056_0036_sept = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_September zein.xlsx")

# okt
rekap_himpunan_0046_0047_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Shania Oktober.xlsx")
rekap_himpunan_0041_0051_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Diah Oktober.xlsx")
rekap_himpunan_2056_2057_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Himpunan 2056-2057 (okt).xlsx")
rekap_himpunan_wa_2060_oktober = pd.read_excel("../data_blast/rekap_himpunan/old/Rekap_Himpunan_2060_Oktober.xlsx")
rekap_himpunan_1097_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Oktiber 1097.xlsx")
rekap_himpunan_7778_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap_Himpunan_Oktober_7778.xlsx")
rekap_himpunan_31_32_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Erni Oktober.xlsx")
rekap_himpunan_1095_1096_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Himpunan Firda Oktober.xlsx")
rekap_himpunan_2058_2059_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Endah Oktober.xlsx")
rekap_himpunan_0034_8196_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap Himpunan Agil Oktober.xlsx")
rekap_himpunan_0056_0036_okt = pd.read_excel("../data_blast/rekap_himpunan/new/Rekap_Himpunan_Oktober zein.xlsx")



rekap_himpunan_wa_2060 = pd.concat([rekap_himpunan_wa_2060_juli,rekap_himpunan_wa_2060_agustus,rekap_himpunan_wa_2060_september,rekap_himpunan_wa_2060_oktober])
rekap_himpunan_wa_0056_0036 = pd.concat([rekap_himpunan_0056_0036_agustus, rekap_himpunan_0056_0036_sept, rekap_himpunan_0056_0036_okt])
rekap_himpunan_wa_2056_2057 = pd.concat([rekap_himpunan_2056_2057_sept,rekap_himpunan_2056_2057_okt])
rekap_himpunan_wa_7778_1097 = pd.concat([rekap_himpunan_wa_7778, rekap_himpunan_wa_7778_Juli, rekap_himpunan_7778_okt,rekap_himpunan_1097_okt])
rekap_himpunan_wa_41_51 = pd.concat([rekap_himpunan_0041_0051_okt])
rekap_himpunan_wa_46_47 = pd.concat([rekap_himpunan_0046_0047_okt])
rekap_himpunan_wa_1095_1096 = pd.concat([rekap_himpunan_1095_1096_okt])
rekap_himpunan_wa_2058_2059 = pd.concat([rekap_himpunan_2058_2059_okt])
rekap_himpunan_wa_31_32 = pd.concat([rekap_himpunan_31_32_okt])
rekap_himpunan_wa_0034_8196 = pd.concat([rekap_himpunan_0034_8196_okt])




# all rekap himpunan
rekap_himpunan = pd.concat([rekap_himpunan_wa_2061,rekap_himpunan_wa_2062, rekap_himpunan_wa_Juni,
                            rekap_himpunan_wa_2060, 
                            rekap_himpunan_wa_0056_0036,
                            rekap_himpunan_wa_2056_2057,
                            rekap_himpunan_wa_7778_1097,
                            rekap_himpunan_wa_41_51,
                            rekap_himpunan_wa_46_47,
                            rekap_himpunan_wa_1095_1096,
                            rekap_himpunan_wa_2058_2059,
                            rekap_himpunan_wa_31_32,
                            rekap_himpunan_wa_0034_8196])


rekap_himpunan['Source'] = "Web Ads"


rekap_himpunan = rekap_himpunan.dropna(subset=['Total'])
rekap_himpunan['Date'] = rekap_himpunan['Date'].fillna(rekap_himpunan['Date Blast']).fillna(rekap_himpunan['Date Donation'])

In [24]:
# CTWA
folder_path = '../data_blast/rekap_himpunan/old/Oktober'

all_files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]

# Read and combine
ctwa_raw = pd.concat(
    [pd.read_excel(os.path.join(folder_path, f)) for f in all_files],
    ignore_index=True
)
ctwa_raw['Source'] = "CTWA"

c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
c:\Users\faruq\AppDa

In [25]:
# rekap_himpunan_all['Whatsapp'] = rekap_himpunan_all['Whatsapp'].replace(['', 'NaN', 'None'], pd.NA)
# Drop rows where Whatsapp is null
ctwa_all = ctwa_raw[['CRM','Tanggal','Nama Donatur','Whatsapps','Payment Status','Bank','Donasi','Akad','Source']].dropna(subset=['Donasi'])
# rekap_himpunan_all
ctwa = ctwa_all.rename(columns={'Whatsapps':'Whatsapp', 'Nama Donatur':'Donatur','Donasi':'Total','Akad':'Program','Tanggal':'Date'})

ctwa

,CRM,Date,Donatur,Whatsapp,Payment Status,Bank,Total,Program,Source
7552,WA 8194,2025-10-20 00:00:00,Nining Suhani,62895327045763.0,Success,BRI,25000.0,Sedekah Palestina,CTWA
7608,WA 8194,2025-10-24 00:00:00,Sonny,6281266171850.0,Success,BSI,50000.0,Sedekah Palestina,CTWA
7629,WA 8194,2025-10-25 00:00:00,Muchtar,6285241131165.0,Success,BNI,25000.0,Sedekah Palestina,CTWA
7664,WA 8194,2025-10-27 00:00:00,Ujarsyah,6285695499458.0,Success,Mandiri,15000.0,Sedekah Palestina,CTWA
7669,WA 8194,2025-10-28 00:00:00,Aji Yudha Dyatmiko,628113700357.0,Success,Mandiri,25000.0,Sedekah Palestina,CTWA
7762,WA 8195,2025-10-20 00:00:00,Surahman Hasty,6285255009899.0,Success,BRI,15000.0,Sedekah Palestina,CTWA
7780,WA 8195,2025-10-21 00:00:00,Ahmad Amali Kurniawan,62818268617.0,Success,BSI,25000.0,Sedekah Palestina,CTWA
7794,WA 8195,2025-10-22 00:00:00,Ambar Kurniati,6281808180400.0,Success,BSI,100000.0,Sedekah Palestina,CTWA
7891,WA 8195,2025-10-27 00:00:00,Eti Rohaeti,62882000680192.0,Success,BSI,50000.0,Sedekah Palestina,CTWA
7957,WA 8196,2025-10-19 00:00:00,Ujang Sanosi,6288218685632.0,Success,Mandiri,25000.0,Sedekah Palestina,CTWA


In [26]:
# make sure whatsapp dtype is the same (int)
ctwa['Whatsapp'] = ctwa['Whatsapp'].astype(int)
rekap_himpunan['Whatsapp'] = rekap_himpunan['Whatsapp'].fillna(0).astype(int)

#  fillna date with date blast
rekap_himpunan['Date'] = rekap_himpunan['Date'].fillna(rekap_himpunan['Date Blast'])

# concat ctwa and rekap himpunan
rekap_himpunan_all = pd.concat([ctwa,rekap_himpunan])

In [27]:
rekap_himpunan_all['Date'] = rekap_himpunan_all['Date'].apply(
    lambda x: pd.to_datetime(x, errors='coerce') if not isinstance(x, pd.Timestamp) else x
)

# choose only necessary columns
rekap_himpunan_all = rekap_himpunan_all[['Donatur', 'Whatsapp', 'Payment Account', 'Date', 'Program',
       'Total', 'Funnel', 'Date Blast', 'Source']]


# get a day column
rekap_himpunan_all['Day'] = rekap_himpunan_all['Date'].dt.strftime('%a')

C:\Users\faruq\AppData\Local\Temp\ipykernel_7308\1903590129.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  lambda x: pd.to_datetime(x, errors='coerce') if not isinstance(x, pd.Timestamp) else x
C:\Users\faruq\AppData\Local\Temp\ipykernel_7308\1903590129.py:2: UserWarning: Parsing dates in %d/%m;%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  lambda x: pd.to_datetime(x, errors='coerce') if not isinstance(x, pd.Timestamp) else x


Standarisasi Payment Account

In [28]:
def payment(akun,number):
    number = str(number)
    if akun == "Yayasan Teman Jalan Kebaikan":
        if ' ' not in number:
            return "BSI"
        else: 
            return number.split()[0]
    else:
        return akun
    
data_all['Payment'] = data_all.apply(lambda row: payment(row['Payment Account'], row['Payment Number']), axis=1)

data_all.drop('Payment Account', axis=1, inplace=True)

In [29]:
def move_column(df, col_name, new_pos):
    cols = list(df.columns)
    cols.insert(new_pos, cols.pop(cols.index(col_name)))
    return df[cols]

data_all = move_column(data_all,"Payment", 13)


data_all.rename(columns={'Payment': 'Payment Account'}, inplace=True)

In [30]:
df = data_all[(data_all['is_random'] == True)]
df = df[df['Payment Status'] == "Success"]
# df = data_all[data_all['Payment Status'] == "Success"]

Menggabungkan data dari web dengan rekap himpunan

In [31]:
rekap_himpunan_all['Program'] = rekap_himpunan_all['Program'].str.upper()

rekap_himpunan_all['Whatsapp'] = rekap_himpunan_all['Whatsapp'].astype(str)

rekap_himpunan_all = rekap_himpunan_all.drop_duplicates()

In [32]:
df = df.loc[:, ~df.columns.str.contains("Unnamed")]
df

,Payment Account,No,Invoice ID,Donatur,Sapaan,Nominal,Kode Unik,Total,Whatsapp,Email,Comment,Program,Payment Method,Payment Number,Payment Status,Fundraiser Commission,Fundraiser Name,Date,Day,Time,UTM Source,UTM Medium,Source,ip,Fundraiser Commision,aff_code,zisco,Data Qurban,Data Package-2,Data Zakat Fitrah,Additional Data,UTM Content,UTM Campaign,UTM Term,UTM ID,hour,label_jam,is_random,kategori_nomor
5055,Mandiri,816.0,INV-2507259LHGG,Rosa Nasution zakrani,Bapak,100000.0,244.0,100244.0,6282288576337,NaN,NaN,SEDEKAH JUMAT PALESTINA,transfer,Mandiri 1410029052222,Success,1002.0,Cekat AI,2025-07-25 00:00:00,Fri,1900-01-01 07:26:00,NaN,NaN,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,07-08,True,Nomor Acak
27037,Mandiri,393.0,INV-2505247ASJW,Metia Rima,Ibu,100000.0,940.0,100940.0,6281933598026,NaN,Angkat penderitaan saudara2 kami di Palestina ...,INFAK PALESTINA,transfer,Mandiri 1410000665448,Success,1009.0,Teman Baik 1,2025-05-24 00:00:00,Sat,1900-01-01 22:52:00,ig,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,22-23,True,Nomor Acak
27038,BNI VA,394.0,INV-250524ESS11,Robert,Kak,100000.0,397.0,103172.0,6285785999179,nk2691785@gmail.com,"Semoga Allah Ridho, aamiin",INFAK PALESTINA,va,9886556100000000000401164,Success,1032.0,Teman Baik 1,2025-05-24 00:00:00,Sat,1900-01-01 22:40:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,22-23,True,Nomor Acak
27041,Mandiri,397.0,INV-250524NAC9J,Hamba ALLOH,Bapak,100000.0,27.0,100000.0,6282378820206,kkasiman68@gmail.com,Semaga ALLOH SWT Memerikan jln keluarSaya tdk ...,INFAK PALESTINA,transfer,Mandiri 1410000665448,Success,1000.0,Teman Baik 1,2025-05-24 00:00:00,Sat,1900-01-01 22:21:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22,22-23,True,Nomor Acak
27042,Mandiri,398.0,INV-250524KQBY4,Tria priliadianto,Ibu,50000.0,555.0,50000.0,6281362597828,triapriliadianto@gmail.com,Mudah mudahan palestina diberikan kekuatan kes...,INFAK PALESTINA,transfer,Mandiri 1410000665448,Success,506.0,Teman Baik 1,2025-05-24 00:00:00,Sat,1900-01-01 21:53:00,fb,paid,Web Ads,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21,21-22,True,Nomor Acak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67974,BCA VA,NaN,INV-251122BI6A0,Santi,NaN,NaN,NaN,53438.0,6285695712425,tintatuh.cute@gmail.com,Yaa Rob lindungi Sudan..,INFAQ SUDAN: BANTUAN KEMANUSIAAN UNTUK SUDAN,va,1900800026182015,Success,NaN,Teman Baik 1,2025-11-22 08:16:30,Sat,1900-01-01 08:16:30,ig,Instagram_Reels,Web Ads,103.28.116.204,534.0,8bmgx,NaN,[],[],[],"{""Kode Unik"":""108""}",Iklan Penjualan Baru,SUDAN 1 TB - Salin,120238002435220693,120238002435190693,8,08-09,True,Nomor Acak
67975,Mandiri,NaN,INV-251122517XR,Desi Andriani,NaN,NaN,NaN,100771.0,628979577192,,Lindungilah saudara2 kami di Sudan Ya Rabb,INFAQ SUDAN: BANTUAN KEMANUSIAAN UNTUK SUDAN,transfer,Mandiri 1410000665448,Success,NaN,Teman Baik 1,2025-11-22 08:29:52,Sat,1900-01-01 08:29:52,ig,Instagram_Reels,Web Ads,114.10.116.121,1008.0,8bmgx,NaN,[],[],[],"{""Kode Unik"":""771""}",Iklan Penjualan Baru,SUDAN 1 TB,120237910001960693,120237910001940693,8,08-09,True,Nomor Acak
67977,GOPAY,NaN,INV-2511220KNS9,Khalishakharinakailakeishakhalael,NaN,NaN,NaN,100354.0,628129948897,,Robbi habli minasholihin\nRobbi habli minashoi...,INFAQ SUDAN: BANTUAN KEMANUSIAAN UNTUK SUDAN,instant,https://gopay.co.id/app/merchanttransfer?tref=...,Success,NaN,Teman Baik 1,2025-11-22 08:31:56,Sat,1900-01-01 08:31:56,ig,Instagram_Stories,Web Ads,180.252.93.73,1004.0,8bmgx,NaN,[],[],[],"{""Kode Unik"":""354""}",Iklan Penjualan Baru,UKB SUDAN TB,120238166431900653,120238166431890653,8,08-09,True,Nomor Acak
67980,BNI,NaN,INV-251122T9U41,Imma,NaN,NaN,NaN,50688.0,6285242060672,,,INFAQ SUDAN: BANTUAN KEMANUSIAAN UNTUK SUDAN,transfer,BNI 3003003257,Success,NaN,Teman Baik 1,2025-11-22 09:26:09,Sat,1900-01-01 09:26:09,ig,Instagram_Feed,Web Ads,180.252.198.175

In [33]:
import re

data_web_himpunan = pd.concat([df,rekap_himpunan_all])

data_web_himpunan['Total'] = data_web_himpunan['Total'].apply(
    lambda x: int(re.sub(r'[^0-9]', '', x)) if isinstance(x, str) else x
)

In [34]:
# Format tanggal dan ambil bulan
data_web_himpunan["Tanggal"] = pd.to_datetime(data_web_himpunan["Date"], dayfirst=True)
data_web_himpunan["Bulan"] = data_web_himpunan["Date"].dt.month
data_web_himpunan["Tahun"] = data_web_himpunan["Date"].dt.year
data_web_himpunan["Bulan_Nama"] = data_web_himpunan["Date"].dt.strftime('%B')

# Bulan pertama donasi
first_donation = data_web_himpunan.groupby("Whatsapp")["Date"].min().reset_index()
first_donation["Bulan_Pertama"] = first_donation["Date"].dt.month
first_donation["Tahun_Pertama"] = first_donation["Date"].dt.year

# Gabung ke df utama
data_web_himpunan = data_web_himpunan.merge(first_donation[["Whatsapp", "Bulan_Pertama"]], on="Whatsapp")
data_web_himpunan = data_web_himpunan[data_web_himpunan["Bulan"] >= data_web_himpunan["Bulan_Pertama"]]

# TRACKING DATA

In [35]:
# Gabungkan data donasi per bulan per Whatsapp
# agg = data_web_himpunan.groupby(["Whatsapp", "Bulan"]).agg({
#     "Donatur": "first",
#     "Total": "sum",
#     "Date": "min",
#     "Payment Account": "first",
#     "Program": lambda x: list(x),
#     "Day":lambda x: x.mode().iloc[0]
# }).reset_index()

agg = data_web_himpunan.groupby(["Whatsapp",  "Bulan"]).agg(
    Donatur=("Donatur", "last"),
    Total=("Total", "sum"),
    Date=("Date", "min"),
    Payment=("Payment Account", "first"),
    Program=("Program", lambda x: list(x)),
    Time=('label_jam', lambda x: list(x)),
    Day=("Day", lambda x: x.mode().iloc[0])
).reset_index()


# Urutkan data untuk pemrosesan berurutan
agg = agg.sort_values(by=["Whatsapp", "Bulan"]).reset_index(drop=True)

In [36]:
# Tambahkan program hasil pemrosesan
final_programs = []
previous_programs = {}

for _, row in agg.iterrows():
    key = row["Whatsapp"]
    current_programs = list(dict.fromkeys(row["Program"]))  # unik, urut
    prev = previous_programs.get(key, [])
    
    # Cari program yang berbeda dari sebelumnya
    different_programs = [p for p in current_programs if p not in prev]

    if not prev:  # bulan pertama
        final_programs.append(current_programs[0])
    elif different_programs:
        final_programs.append(different_programs[0])  # tampilkan program baru
    else:
        final_programs.append(current_programs[0])  # sama semua, tampilkan salah satu

    previous_programs[key] = current_programs

In [37]:
from collections import Counter

# Example: assuming df is your DataFrame
def custom_mode_from_list(time_list):
    if not time_list or len(time_list) == 0:
        return None
    if len(time_list) == 1:
        return time_list[0]
    
    counter = Counter(time_list)
    most_common = counter.most_common()
    
    if len(most_common) > 1 and most_common[0][1] == most_common[1][1]:
        return time_list[-1]  # Tie: return last value
    return most_common[0][0]  # Clear mode

# Make sure the "Time" column is a list (not a string)
# If it's stringified, convert it using ast.literal_eval
import ast
agg['Time'] = agg['Time'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Apply the function to each row
agg['Label_Jam'] = agg['Time'].apply(custom_mode_from_list)

agg["Final_Program"] = final_programs

In [38]:
# Persiapan output format lebar (wide)
bulan_dict = {
    1: "Januari", 2: "Februari", 3: "Maret", 4: "April", 5: "Mei", 6: "Juni",
    7: "Juli", 8: "Agustus", 9: "September", 10: "Oktober", 11: "November", 12: "Desember"
}

# Output awal
output = pd.DataFrame(agg["Whatsapp"].drop_duplicates())

for bulan_num in range(1, 13):
    bulan_nama = bulan_dict[bulan_num]
    sub = agg[agg["Bulan"] == bulan_num][["Whatsapp", "Final_Program", "Total", "Date", "Day", "Label_Jam", "Payment"]]
    sub.columns = ["Whatsapp", f"{bulan_nama}_Program", f"{bulan_nama}_Donasi", f"{bulan_nama}_Tanggal", f"{bulan_nama}_Day", f"{bulan_nama}_Time", f"{bulan_nama}_Payment"]
    output = output.merge(sub, on="Whatsapp", how="left")

# Tambah Nama
output = output.merge(data_web_himpunan[["Whatsapp", "Donatur"]].drop_duplicates("Whatsapp"), on="Whatsapp", how="left")

# Tambah bulan pertama donasi untuk pengurutan
output = output.merge(first_donation[["Whatsapp", "Bulan_Pertama"]], on="Whatsapp")
output = output.sort_values("Bulan_Pertama").drop(columns=["Bulan_Pertama"])

# Urutkan kolom
cols = output.columns.tolist()
cols = ["Whatsapp", "Donatur"] + [col for col in cols if col not in ["Whatsapp", "Donatur"]]
output = output[cols]

# Hapus nilai 0
output.replace(0, "", inplace=True)

In [39]:
# Ambil semua kolom yang mengandung '_Program'
program_cols = [col for col in output.columns if '_Program' in col]

time_cols = [col for col in output.columns if '_Time' in col]

In [40]:
# Total donasi
output['Total'] = output[[col for col in output.columns if '_Donasi' in col]].sum(axis=1)

# Frekuensi donasi (jumlah kolom donasi yang tidak kosong)
output['Frekuensi'] = output[[col for col in output.columns if '_Donasi' in col]].notna().sum(axis=1)

# output['Total_Preferensi'] = (output[[col for col in output.columns if '_Preferensi' in col]].sum(axis=1) / output[[col for col in output.columns if '_Preferensi' in col]].notna().sum(axis=1)).astype(int)

# Hitung preferensi: jumlah program unik per baris
output['Preferensi'] = output[program_cols].apply(lambda row: row.dropna().nunique(), axis=1)

# Hitung program yang paling sering muncul per baris
output['Program'] = output[program_cols].apply(lambda row: row.dropna().mode().iloc[0] if not row.dropna().empty else None, axis=1)

# Hitung range time paling sering muncul per baris
output['Label_Jam'] = output[time_cols].apply(lambda row: row.dropna().mode().iloc[0] if not row.dropna().empty else None, axis=1)

# Rata-rata donasi, dibulatkan ke bilangan bulat
output['Rata - rata'] = output[[col for col in output.columns if '_Donasi' in col]].mean(axis=1).round(0).astype(int)

# Change total dtype to int
output['Total'] = output['Total'].astype(int)

# Find Day Mode and date catogory in each row

In [41]:
date_and_day = data_web_himpunan.groupby('Whatsapp').agg(
    Day_Mode=("Day", lambda x: x.mode().iloc[0]),
    Tanggal_terakhir_donasi=("Tanggal", max),
    Date=("Tanggal", list),
    Source=("Source","first")
).reset_index()

C:\Users\faruq\AppData\Local\Temp\ipykernel_7308\1344697317.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  date_and_day = data_web_himpunan.groupby('Whatsapp').agg(


In [42]:
def classify_cutoff(dates):
    # Konversi ke pandas Series dan ambil hari (tanggal) saja
    days = pd.Series(dates).dt.day
    
    # Hitung modus
    modus = days.mode().iloc[0]
    
    # Klasifikasi berdasarkan rentang tanggal cut-off
    return 'Masa Gajian' if (modus >= 26 or modus <= 5) else 'Bukan Gajian'

date_and_day['Date_Category'] = date_and_day['Date'].apply(classify_cutoff)
date_and_day['Date_Category'] = np.where(date_and_day['Day_Mode'] == 'Fri', 'Jumat', date_and_day['Date_Category'])
date_and_day = date_and_day.drop(columns=['Date'])

In [43]:
# merge date and day and also first donation
output = pd.merge(output,date_and_day, how='left', on='Whatsapp')
output = output.merge(first_donation, how='left', on='Whatsapp')

# change the year and month from date column
output['Bulan'] = output['Date'].dt.strftime('%B')
output['Tahun'] = output['Date'].dt.year

In [44]:
# 
output['klasifikasi_program'] = np.where(
    output['Program'].str.lower().str.contains("palestina|gaza"),
    "PALESTINA",
    np.where(
        output['Program'].str.lower().str.contains("sudan"),
        "SUDAN",
        output['Program']
    )
)

# Tambahkan kolom 'Kategori' berdasarkan nilai 'Frekuensi'
output['Kategori'] = output['Frekuensi'].apply(
    lambda x: 'Pasif' if x < 1 else ('Aktif' if x < 4.5 else 'Loyal')
)

# Tambahkan kolom 'Badge' berdasarkan nilai 'Total'
output['Badge'] = output['Total'].apply(
    lambda x: 'BRONZE' if x < 100000 else ('SILVER' if x <= 1000000 else ('GOLD' if x < 5000000 else 'PLATINUM'))
)

# Tambahkan kolom 'Avg Kategori' berdasarkan nilai 'Rata - rata'
output['Avg Kategori'] = output['Rata - rata'].apply(
    lambda x: "< 100.000" if x < 100000 else ("100.000 - 1000.000" if x <= 1000000 else ("1000.000 - 5000.000" if x < 5000000 else "> 5000.000"))
)

In [45]:
def update_data_himpunan(main_df, updated_df, cols_to_check, key_col="Whatsapp"):

    # --- 0️⃣ Remove unnamed columns to prevent alignment issues
    main_df = main_df.loc[:, ~main_df.columns.str.contains('^Unnamed')]
    updated_df = updated_df.loc[:, ~updated_df.columns.str.contains('^Unnamed')]

    # --- 1️⃣ Ensure the key column exists
    if key_col not in main_df.columns or key_col not in updated_df.columns:
        raise KeyError(f"❌ Key column '{key_col}' not found in one of the DataFrames")

    # --- 2️⃣ Set index to key column
    main_df = main_df.set_index(key_col, drop=False)
    updated_df = updated_df.set_index(key_col, drop=False)

    # --- 3️⃣ Keep only common columns
    updated_df = updated_df.loc[:, updated_df.columns.intersection(main_df.columns)]

    # --- 4️⃣ Align index based on key column
    common_ids = updated_df.index.intersection(main_df.index)

    # --- 5️⃣ Match dtypes
    for col in main_df.columns.intersection(updated_df.columns):
        if main_df[col].dtype != updated_df[col].dtype:
            try:
                updated_df[col] = updated_df[col].astype(main_df[col].dtype)
            except Exception:
                if pd.api.types.is_datetime64_any_dtype(main_df[col]):
                    updated_df[col] = pd.to_datetime(updated_df[col], errors='coerce')
                elif pd.api.types.is_numeric_dtype(main_df[col]):
                    updated_df[col] = pd.to_numeric(updated_df[col], errors='coerce')
                else:
                    updated_df[col] = updated_df[col].astype(str)

    # --- 6️⃣ Detect differences
    diff_mask = updated_df.loc[common_ids, cols_to_check].ne(
        main_df.loc[common_ids, cols_to_check]
    ).any(axis=1)

    # --- 7️⃣ Update only changed rows
    cols_to_update = updated_df.columns.intersection(main_df.columns)
    main_df.loc[common_ids[diff_mask], cols_to_update] = updated_df.loc[common_ids[diff_mask], cols_to_update].values

    # --- 8️⃣ Add new rows (new WhatsApp IDs)
    new_ids = updated_df.index.difference(main_df.index)
    main_df = pd.concat([main_df, updated_df.loc[new_ids, cols_to_update]])

    print("✅ Excel updated successfully:")
    print(f"- {len(new_ids)} new rows added")
    print(f"- {diff_mask.sum()} rows updated based on {cols_to_check}")

    return main_df


In [46]:
data_with_rekapan["Whatsapp"] = data_with_rekapan["Whatsapp"].astype(str)

In [47]:
# data_with_rekapan_updated = update_dataframe(
#     main_df=data_with_rekapan,
#     updated_df=output,
#     cols_to_check=["Total","Badge"]
# )

data_with_rekapan_updated = update_data_web(
    main_df=data_with_rekapan,
    updated_df=output,
    key_col="Whatsapp",
    cols_to_check=["Total","Date_Category","Source", "klasifikasi_program"]
)

C:\Users\faruq\AppData\Local\Temp\ipykernel_7308\871079552.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  main_df.loc[rows_to_overwrite.index] = rows_to_overwrite
C:\Users\faruq\AppData\Local\Temp\ipykernel_7308\871079552.py:73: FutureWarning: Setting an item of incompa

✅ Overwrite update completed:
- 135 rows overwritten (different in ['Total', 'Date_Category', 'Source', 'klasifikasi_program'])
- 98 new rows added
- Final total rows: 45246


In [48]:
duplicates = data_with_rekapan_updated[data_with_rekapan_updated['Whatsapp'].duplicated(keep=False)]

if not duplicates.empty:
    print("❌ Duplicate rows found:\n", duplicates)
    raise SystemExit("Stopping cell due to duplicates.")

In [49]:
data_with_rekapan_updated = data_with_rekapan_updated[['Whatsapp', 'Donatur',
       'Januari_Program', 'Januari_Donasi', 'Januari_Tanggal', 'Januari_Day',
       'Januari_Time', 'Januari_Payment', 'Februari_Program',
       'Februari_Donasi', 'Februari_Tanggal', 'Februari_Day', 'Februari_Time',
       'Februari_Payment', 'Maret_Program', 'Maret_Donasi', 'Maret_Tanggal',
       'Maret_Day', 'Maret_Time', 'Maret_Payment', 'April_Program',
       'April_Donasi', 'April_Tanggal', 'April_Day', 'April_Time',
       'April_Payment', 'Mei_Program', 'Mei_Donasi', 'Mei_Tanggal', 'Mei_Day',
       'Mei_Time', 'Mei_Payment', 'Juni_Program', 'Juni_Donasi',
       'Juni_Tanggal', 'Juni_Day', 'Juni_Time', 'Juni_Payment', 'Juli_Program',
       'Juli_Donasi', 'Juli_Tanggal', 'Juli_Day', 'Juli_Time', 'Juli_Payment',
       'Agustus_Program', 'Agustus_Donasi', 'Agustus_Tanggal', 'Agustus_Day',
       'Agustus_Time', 'Agustus_Payment', 'September_Program',
       'September_Donasi', 'September_Tanggal', 'September_Day',
       'September_Time', 'September_Payment', 'Oktober_Program',
       'Oktober_Donasi', 'Oktober_Tanggal', 'Oktober_Day', 'Oktober_Time',
       'Oktober_Payment', 'November_Program', 'November_Donasi',
       'November_Tanggal', 'November_Day', 'November_Time', 'November_Payment',
       'Desember_Program', 'Desember_Donasi', 'Desember_Tanggal',
       'Desember_Day', 'Desember_Time', 'Desember_Payment', 'Total',
       'Frekuensi', 'Preferensi', 'Program','klasifikasi_program','Label_Jam', 'Rata - rata',
       'Day_Mode', 'Date_Category', 'Date', 'Bulan_Pertama', 'Tahun_Pertama',
       'Bulan', 'Tahun', 'Kategori', 'Badge', 'Avg Kategori',"Tanggal_terakhir_donasi", "Source"]]

data_with_rekapan_updated.to_excel("checkpoint_files/data_with_updated_himpunan.xlsx", index=False)

# Check number validity

In [63]:
# data with number validity (registered and not)
data_with_validity = pd.read_excel("checkpoint_files/data_with_number_validity_updated.xlsx")
data_with_validity = data_with_validity[["Whatsapp", "validity"]]

# crm data
df_crm = pd.read_excel("checkpoint_files/data_each_crm_updated.xlsx")
df_crm = df_crm[['Whatsapp','CRM']]

# category each crm
df_category = pd.read_excel("checkpoint_files/category_each_donor.xlsx")

# make sure whatsapp dtype is int on both dataframes
df_crm['Whatsapp'] = df_crm['Whatsapp'].astype(int)
data_with_rekapan_updated['Whatsapp'] = data_with_rekapan_updated['Whatsapp'].astype(int)
data_with_validity['Whatsapp'] = data_with_validity['Whatsapp'].astype(int)

# merge to get a number validity
df_tracking_valid = data_with_rekapan_updated.merge(data_with_validity, how="left", on="Whatsapp")


df_with_crm = df_tracking_valid.merge(df_crm,how='left',on='Whatsapp')

In [64]:
df_tracking_valid = df_tracking_valid[['Whatsapp', 'Donatur', 'Bulan', 'Tahun', 'Januari_Program', 'Januari_Donasi',
       'Januari_Tanggal', 'Januari_Day', 'Januari_Time', 'Januari_Payment',
       'Februari_Program', 'Februari_Donasi', 'Februari_Tanggal',
       'Februari_Day', 'Februari_Time', 'Februari_Payment', 'Maret_Program',
       'Maret_Donasi', 'Maret_Tanggal', 'Maret_Day', 'Maret_Time',
       'Maret_Payment', 'April_Program', 'April_Donasi', 'April_Tanggal',
       'April_Day', 'April_Time', 'April_Payment', 'Mei_Program', 'Mei_Donasi',
       'Mei_Tanggal', 'Mei_Day', 'Mei_Time', 'Mei_Payment', 'Juni_Program',
       'Juni_Donasi', 'Juni_Tanggal', 'Juni_Day', 'Juni_Time', 'Juni_Payment',
       'Juli_Program', 'Juli_Donasi', 'Juli_Tanggal', 'Juli_Day', 'Juli_Time',
       'Juli_Payment', 'Agustus_Program', 'Agustus_Donasi', 'Agustus_Tanggal',
       'Agustus_Day', 'Agustus_Time', 'Agustus_Payment', 'September_Program',
       'September_Donasi', 'September_Tanggal', 'September_Day',
       'September_Time', 'September_Payment', 'Oktober_Program',
       'Oktober_Donasi', 'Oktober_Tanggal', 'Oktober_Day', 'Oktober_Time',
       'Oktober_Payment', 'November_Program', 'November_Donasi',
       'November_Tanggal', 'November_Day', 'November_Time', 'November_Payment',
       'Desember_Program', 'Desember_Donasi', 'Desember_Tanggal',
       'Desember_Day', 'Desember_Time', 'Desember_Payment', 'Total',
       'Frekuensi', 'Preferensi', 'Program','klasifikasi_program' ,'Label_Jam', 'Rata - rata',
       'Day_Mode', 'Date_Category', 'Date', 'Bulan_Pertama', 'Tahun_Pertama',
       'Kategori', 'Badge', 'Avg Kategori','Tanggal_terakhir_donasi', 'validity', 'Source']]

In [65]:
duplicates = df_tracking_valid[df_tracking_valid['Whatsapp'].duplicated(keep=False)]

if not duplicates.empty:
    print("❌ Duplicate rows found:\n", duplicates)
    raise SystemExit("Stopping cell due to duplicates.")

In [66]:
df_with_crm = df_tracking_valid.merge(df_crm,how='left',on='Whatsapp')

321789732189

In [67]:
df_with_crm = df_with_crm[['Whatsapp', 'Donatur',  'CRM', 'Bulan', 'Tahun', 'Januari_Program',
       'Januari_Donasi', 'Januari_Tanggal', 'Januari_Day', 'Januari_Time',
       'Januari_Payment', 'Februari_Program', 'Februari_Donasi',
       'Februari_Tanggal', 'Februari_Day', 'Februari_Time', 'Februari_Payment',
       'Maret_Program', 'Maret_Donasi', 'Maret_Tanggal', 'Maret_Day',
       'Maret_Time', 'Maret_Payment', 'April_Program', 'April_Donasi',
       'April_Tanggal', 'April_Day', 'April_Time', 'April_Payment',
       'Mei_Program', 'Mei_Donasi', 'Mei_Tanggal', 'Mei_Day', 'Mei_Time',
       'Mei_Payment', 'Juni_Program', 'Juni_Donasi', 'Juni_Tanggal',
       'Juni_Day', 'Juni_Time', 'Juni_Payment', 'Juli_Program', 'Juli_Donasi',
       'Juli_Tanggal', 'Juli_Day', 'Juli_Time', 'Juli_Payment',
       'Agustus_Program', 'Agustus_Donasi', 'Agustus_Tanggal', 'Agustus_Day',
       'Agustus_Time', 'Agustus_Payment', 'September_Program',
       'September_Donasi', 'September_Tanggal', 'September_Day',
       'September_Time', 'September_Payment', 'Oktober_Program',
       'Oktober_Donasi', 'Oktober_Tanggal', 'Oktober_Day', 'Oktober_Time',
       'Oktober_Payment', 'November_Program', 'November_Donasi',
       'November_Tanggal', 'November_Day', 'November_Time', 'November_Payment',
       'Desember_Program', 'Desember_Donasi', 'Desember_Tanggal',
       'Desember_Day', 'Desember_Time', 'Desember_Payment', 'Total',
       'Frekuensi', 'Preferensi', 'Program', 'klasifikasi_program',
       'Label_Jam', 'Rata - rata', 'Day_Mode', 'Date_Category', 'Date',
       'Bulan_Pertama', 'Tahun_Pertama', 'Kategori', 'Badge', 'Avg Kategori',
       'Tanggal_terakhir_donasi', 'validity', 'Source']]

In [68]:
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
df_with_crm['Bulan'] = pd.Categorical(df_with_crm['Bulan'], categories=month_order, ordered=True)
df_with_crm = df_with_crm.sort_values('Bulan')

In [69]:
duplicates = df_with_crm[df_with_crm['Whatsapp'].duplicated(keep=False)]

if not duplicates.empty:
    print("❌ Duplicate rows found:\n", duplicates)
    raise SystemExit("Stopping cell due to duplicates.")

In [70]:
df_with_crm['validity'].value_counts(dropna=False)

validity
NaN                                                                                                                                                23539
Number registered                                                                                                                                  19077
Number not registered                                                                                                                               2608
Error: 504 Server Error: Gateway Time-out for url: https://api.starsender.online/api/check-number                                                     14
Error: 502 Server Error: Bad Gateway for url: https://api.starsender.online/api/check-number                                                           4
Error: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))        3
Timeout                                                                  

In [71]:
df_with_crm.to_excel('Tracking_temanbaik.xlsx')

In [72]:
df_summary_database = df_with_crm[['Whatsapp',
 'Donatur',
 'Bulan',
 'Tahun',
 'CRM',
 'Source',
 'Total',
 'Frekuensi',
 'klasifikasi_program',
 'Preferensi',
 'Label_Jam',
 'Rata - rata',
 'Day_Mode',
 'Date_Category',
 'Tahun_Pertama',
 'Kategori',
 'Badge',
 'Avg Kategori', 'validity']]

In [73]:
df = df_summary_database[(df_summary_database['CRM'].isna()) & (df_summary_database['Source'] == "Web Ads")]
df[df['validity'].isna()]

,Whatsapp,Donatur,Bulan,Tahun,CRM,Source,Total,Frekuensi,klasifikasi_program,Preferensi,Label_Jam,Rata - rata,Day_Mode,Date_Category,Tahun_Pertama,Kategori,Badge,Avg Kategori,validity


In [74]:
df_summary_database.to_excel('Database.xlsx')

In [62]:
data_with_rekapan_updated['Katasdfkljalskdjfri'].value_sadcounts(dropna=False)

KeyError: 'Katasdfkljalskdjfri'

# Data each crm

In [ ]:
data_with_rekapan_updated['Kategori'].value_counts(dropna=False)

NameError: name 'data_with_rekapan_updated' is not defined

In [ ]:
tracking_with_crm = pd.read_excel("checkpoint_files/data_tracking_with_crm.xlsx")

In [ ]:
# df_7778_Intan = pd.read_excel("../data_blast/data_each_crm/data_WA_7778_Intan.xlsx")
# df_1097_Intan = pd.read_excel("../data_blast/data_each_crm/data_WA_1097_Intan.xlsx")
# df_0036_zein = pd.read_excel("../data_blast/data_each_crm/data_WA_0036_Zein.xlsx")
# df_0056_zein = pd.read_excel("../data_blast/data_each_crm/data_WA_0056_Zein.xlsx")
# df_0041_Diah = pd.read_excel("../data_blast/data_each_crm/data_WA_0041_Diah.xlsx")
# df_0051_Diah = pd.read_excel("../data_blast/data_each_crm/data_WA_0051_Diah.xlsx")
# df_0047_Shania = pd.read_excel("../data_blast/data_each_crm/data_WA_0047_Shania.xlsx")
# df_0046_Shania = pd.read_excel("../data_blast/data_each_crm/data_WA_0046_Shania.xlsx")
# df_1096_Firda = pd.read_excel("../data_blast/data_each_crm/data_WA_1096_Firda.xlsx")
# df_2059_Endah = pd.read_excel("../data_blast/data_each_crm/data_WA_2059_Endah.xlsx")
# df_2058_Endah = pd.read_excel("../data_blast/data_each_crm/data_WA_2058_Endah.xlsx")
# df_1095_Firda = pd.read_excel("../data_blast/data_each_crm/data_WA_1095_Firda.xlsx")
# df_2057_vicky = pd.read_excel("../data_blast/data_each_crm/data_WA_2057_Vicky.xlsx")
# df_2056_vicky = pd.read_excel("../data_blast/data_each_crm/data_WA_2056_Vicky.xlsx")
# df_0031_Erni = pd.read_excel("../data_blast/data_each_crm/data_WA_0031_Erni.xlsx")
# df_0032_Erni = pd.read_excel("../data_blast/data_each_crm/data_WA_0032_Erni.xlsx")
# df_0034_Agil = pd.read_excel("../data_blast/data_each_crm/data_WA_0034_Agil.xlsx")
# df_1094_Agil = pd.read_excel("../data_blast/data_each_crm/data_WA_1094_Agil.xlsx")
# df_2062 = pd.read_excel("../data_blast/data_each_crm/data_WA_2062.xlsx")
# df_2061 = pd.read_excel("../data_blast/data_each_crm/data_WA_2061.xlsx")
# df_2060 = pd.read_excel("../data_blast/data_each_crm/data_WA_2060.xlsx")
# df_nothing = pd.read_excel("../data_blast/data_each_crm/data_harus_tanpa_CRM.xlsx")
# refill_1097 = pd.read_excel("../data_blast/data_each_crm/Intan/refill_data_WA_1097_Intan.xlsx")
# refill_2056 = pd.read_excel("../data_blast/data_each_crm/Vicky/refill_data_WA_2056_Vicky.xlsx")
# refill_2057 = pd.read_excel("../data_blast/data_each_crm/Vicky/refill_data_WA_2057_Vicky.xlsx")
# df_olivia_1 = pd.read_excel("../data_blast/data_each_crm/Olivia/data_WA_Olivia_1.xlsx")
# df_olivia_2 = pd.read_excel("../data_blast/data_each_crm/Olivia/data_WA_Olivia_2.xlsx")

In [ ]:
# df_crm = pd.concat([df_7778_Intan, df_2062, df_2061, df_2060, df_nothing, df_0036_zein, df_0056_zein, df_2056_vicky, df_2057_vicky, df_1097_Intan, df_0041_Diah, df_0051_Diah, df_1095_Firda, df_1096_Firda, df_2058_Endah, df_2059_Endah, df_0046_Shania, df_0047_Shania, df_0031_Erni, df_0032_Erni,df_0034_Agil,df_1094_Agil])

In [ ]:
df_crm = pd.read_excel("checkpoint_files/data_each_crm_updated.xlsx")

NameError: name 'pd' is not defined

In [ ]:
df_crm['Whatsapp'] = df_crm['Whatsapp'].astype(str)
df_crm['CRM'].value_counts()

CRM
WA_2060        10674
WA_7778         2961
WA_0036         2500
WA_0056         2000
WA_2056         2000
WA_2057         2000
WA_1097         1500
Eliminated      1477
WA_1095          889
WA_0046          863
WA_Olivia        863
WA_1094          863
WA_0034          863
WA_0032          863
WA_0031          863
WA_0047          863
WA_0051          863
WA_2059          863
WA_2058          863
WA_1096          863
WA_0041          863
WA_Olivia_2      863
WA_2062          144
WA_2061          141
nothing            1
Name: count, dtype: int64

merge data on tracking with data each crm

In [ ]:
df_crm = df_crm[['Whatsapp','CRM',
       'Donatur']]

In [ ]:
data_with_rekapan_updated['Whatsapp'] = data_with_rekapan_updated['Whatsapp'].astype(str)

In [ ]:
df_crm['Whatsapp'] = df_crm['Whatsapp'].astype(str)

In [ ]:
df_final = data_with_rekapan_updated.merge(df_crm, how='left', on='Whatsapp')

In [ ]:
df_final['CRM'].value_counts()

CRM
WA_2060        10674
WA_7778         2961
WA_0036         2500
WA_2056         2000
WA_0056         2000
WA_2057         2000
WA_1097         1500
Eliminated      1477
WA_1095          889
WA_2058          863
WA_Olivia        863
WA_0034          863
WA_0031          863
WA_1096          863
WA_2059          863
WA_1094          863
WA_0046          863
WA_0032          863
WA_0047          863
WA_0051          863
WA_0041          863
WA_Olivia_2      863
WA_2062          144
WA_2061          140
nothing            1
Name: count, dtype: int64

In [ ]:
# Contoh: pindahkan kolom 'email' ke posisi ke-1 (indeks 0 berarti paling depan)
df_final = move_column(df_final, 'Bulan', 2)
df_final = move_column(df_final, 'Tahun', 3)
df_final = move_column(df_final, "CRM", 4)

In [ ]:
df_final['klasifikasi_program'] = np.where(df_final['Program'].str.lower().str.contains("palestina|gaza"), "PALESTINA", df_final['Program'])

In [ ]:
df_final = df_final.rename(columns={"Donatur_x":"Donatur"})

In [ ]:
df_final = df_final.drop(['Date','Bulan_Pertama','Donatur_y', 'Program'], axis=1)

In [ ]:
df_final = move_column(df_final, "klasifikasi_program", 79)

In [ ]:
df_final['Whatsapp'] = df_final['Whatsapp'].astype(int)

In [ ]:
# def update_crm(main_df, updated_df, cols_to_check, log_limit=10):
#     """
#     Update main_df with updated_df using index as ID (Whatsapp).
#     - Only updates selected columns (cols_to_check)
#     - Does NOT overwrite with NaN
#     - Inserts new rows if not exist
#     - Prints log of changes (limited)
#     """
#     main_df = main_df.copy()
#     updated_df = updated_df.copy()

#     # Pastikan kolom yang dicek ada di kedua dataframe
#     for col in cols_to_check:
#         if col not in updated_df.columns or col not in main_df.columns:
#             raise KeyError(f"Kolom '{col}' tidak ada di kedua dataframe.")

#     # Temukan index yang sama
#     common_ids = updated_df.index.intersection(main_df.index)

#     # Update rows jika ada perbedaan
#     changes_log = []
#     for idx in common_ids:
#         for col in cols_to_check:
#             old_val = main_df.at[idx, col]
#             new_val = updated_df.at[idx, col]

#             # Hanya update kalau beda dan new_val TIDAK NaN
#             if pd.notna(new_val) and old_val != new_val:
#                 main_df.at[idx, col] = new_val
#                 changes_log.append(f"[UPDATE] {idx} | {col}: '{old_val}' → '{new_val}'")

#     # Insert rows baru
#     new_rows = updated_df.loc[~updated_df.index.isin(main_df.index)]
#     main_df = pd.concat([main_df, new_rows], axis=0)

#     # Print log perubahan
#     print("===== LOG PERUBAHAN DATA =====")
#     if changes_log:
#         if len(changes_log) > log_limit:
#             print("\n".join(changes_log[:log_limit]))
#             print(f"... and {len(changes_log) - log_limit} more changes")
#         else:
#             print("\n".join(changes_log))
#     else:
#         print("Tidak ada data yang berubah.")
#     print(f"\nBaris baru ditambahkan: {len(new_rows)}")

#     return main_df
def update_status_dataframe(main_df: pd.DataFrame, updated_df: pd.DataFrame, key_col: str, cols_to_update: list) -> pd.DataFrame:
    """
    Update main_df using updated_df based on a key column and selected columns.
    - Updates values for matching rows.
    - Adds new rows if key not found in main_df.
    - Adds new columns from updated_df if they don't exist in main_df.
    """

    # --- 0️⃣ Copy data to avoid modifying originals ---
    main_df = main_df.copy()
    updated_df = updated_df.copy()

    # --- 1️⃣ Handle duplicate keys ---
    if main_df[key_col].duplicated().any():
        print(f"⚠️ Warning: Duplicate keys found in main_df on column '{key_col}' — keeping first occurrence.")
        main_df = main_df.drop_duplicates(subset=key_col, keep='first')

    if updated_df[key_col].duplicated().any():
        print(f"⚠️ Warning: Duplicate keys found in updated_df on column '{key_col}' — keeping first occurrence.")
        updated_df = updated_df.drop_duplicates(subset=key_col, keep='first')

    # --- 2️⃣ Ensure new columns from updated_df exist in main_df ---
    new_columns = [col for col in updated_df.columns if col not in main_df.columns]
    if new_columns:
        print(f"🆕 Found new columns in updated_df: {new_columns}")
        for col in new_columns:
            main_df[col] = None  # initialize with None (or pd.NA)

    # --- 3️⃣ Set index for efficient update ---
    main_df = main_df.set_index(key_col)
    updated_df = updated_df.set_index(key_col)

    # --- 4️⃣ Identify common and new keys ---
    common_ids = updated_df.index.intersection(main_df.index)
    new_rows = updated_df.loc[~updated_df.index.isin(main_df.index)]

    # --- 5️⃣ Update selected columns ---
    for col in cols_to_update:
        if col in updated_df.columns:
            main_df.loc[common_ids, col] = updated_df.loc[common_ids, col]
        else:
            print(f"⚠️ Column '{col}' not found in updated_df — skipped.")

    # --- 6️⃣ Add new rows ---
    updated_main_df = pd.concat([main_df, new_rows])

    # --- 7️⃣ Reset index back ---
    updated_main_df = updated_main_df.reset_index()

    # --- 8️⃣ Log summary ---
    print("✅ Update process completed successfully:")
    print(f"- {len(common_ids)} matched rows updated ({cols_to_update})")
    print(f"- {len(new_rows)} new rows added")
    print(f"- {len(new_columns)} new columns added")
    print(f"- Final total rows: {len(updated_main_df)}")

    return updated_main_df



In [ ]:
df_crm['Whatsapp'] = df_crm['Whatsapp'].astype(int)

In [ ]:
tracking_with_crm_updated = update_status_dataframe(
    main_df=tracking_with_crm,
    updated_df=df_final,
    key_col="Whatsapp",
    cols_to_update=["Tanggal_terakhir_donasi",'Kategori']
)

C:\Users\faruq\AppData\Local\Temp\ipykernel_12308\1363957487.py:93: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  updated_main_df = pd.concat([main_df, new_rows])


✅ Update process completed successfully:
- 43503 matched rows updated (['Tanggal_terakhir_donasi', 'Kategori'])
- 98 new rows added
- 0 new columns added
- Final total rows: 43602


In [ ]:
tracking_with_crm_updated = tracking_with_crm_updated.reset_index(drop=True)

In [ ]:
duplicates = tracking_with_crm_updated[tracking_with_crm_updated['Whatsapp'].duplicated(keep=False)]

if not duplicates.empty:
    print("❌ Duplicate rows found:\n", duplicates)
    raise SystemExit("Stopping cell due to duplicates.")

In [ ]:
tracking_with_crm_updated = tracking_with_crm_updated[['Whatsapp', 'Donatur', 'Bulan', 'Tahun', 'CRM',
       'Januari_Program', 'Januari_Donasi', 'Januari_Tanggal', 'Januari_Day',
       'Januari_Time', 'Januari_Payment', 'Februari_Program',
       'Februari_Donasi', 'Februari_Tanggal', 'Februari_Day', 'Februari_Time',
       'Februari_Payment', 'Maret_Program', 'Maret_Donasi', 'Maret_Tanggal',
       'Maret_Day', 'Maret_Time', 'Maret_Payment', 'April_Program',
       'April_Donasi', 'April_Tanggal', 'April_Day', 'April_Time',
       'April_Payment', 'Mei_Program', 'Mei_Donasi', 'Mei_Tanggal', 'Mei_Day',
       'Mei_Time', 'Mei_Payment', 'Juni_Program', 'Juni_Donasi',
       'Juni_Tanggal', 'Juni_Day', 'Juni_Time', 'Juni_Payment', 'Juli_Program',
       'Juli_Donasi', 'Juli_Tanggal', 'Juli_Day', 'Juli_Time', 'Juli_Payment',
       'Agustus_Program', 'Agustus_Donasi', 'Agustus_Tanggal', 'Agustus_Day',
       'Agustus_Time', 'Agustus_Payment', 'September_Program',
       'September_Donasi', 'September_Tanggal', 'September_Day',
       'September_Time', 'September_Payment', 'Oktober_Program',
       'Oktober_Donasi', 'Oktober_Tanggal', 'Oktober_Day', 'Oktober_Time',
       'Oktober_Payment', 'November_Program', 'November_Donasi',
       'November_Tanggal', 'November_Day', 'November_Time', 'November_Payment',
       'Desember_Program', 'Desember_Donasi', 'Desember_Tanggal',
       'Desember_Day', 'Desember_Time', 'Desember_Payment', 'Total',
       'Frekuensi', 'klasifikasi_program', 'Preferensi', 'Label_Jam',
       'Rata - rata', 'Day_Mode', 'Date_Category', 'Tahun_Pertama', 'Kategori',
       'Badge', 'Avg Kategori',"Tanggal_terakhir_donasi"]]
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']
tracking_with_crm_updated['Bulan'] = pd.Categorical(tracking_with_crm_updated['Bulan'], categories=month_order, ordered=True)
tracking_with_crm_updated = tracking_with_crm_updated.sort_values('Bulan')

In [ ]:
tracking_with_crm_updated.to_excel("checkpoint_files/data_tracking_with_crm.xlsx")

# Daily activity each crm

In [ ]:
tracking_with_status = pd.read_excel("checkpoint_files/tracking_with_updated_category_daily_activity.xlsx")

In [ ]:
status_donor = tracking_with_status[['Whatsapp','Kategori']]

here is to add the updated daily activity to update the donor category

In [ ]:
status_donor = tracking_with_crm_updated.merge(status_donor,how='left',on='Whatsapp')

In [ ]:
tracking_baru = status_donor
status_donor = status_donor[["Whatsapp","Kategori_y"]]

In [ ]:
daily_1097 = pd.read_excel("../data_blast/daily_activity/filled/new/Daily_Activity_1097.xlsx")
daily_2056_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2056 (BONDING).xlsx")
daily_2056_maintain = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2056 (MAINTAIN).xlsx")
daily_2057_bonding = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2057 (BONDING).xlsx")
daily_2057_maintain = pd.read_excel("../data_blast/daily_activity/filled/new/Daily Activity 2057 (MAINTAIN).xlsx")

c:\Users\faruq\AppData\Local\Programs\Python\Python313\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
daily_2056 = pd.concat([daily_2056_bonding, daily_2056_maintain])
daily_2057 = pd.concat([daily_2057_bonding, daily_2057_maintain])

 - daily that has been tracked

In [ ]:
daily_activity_all = pd.concat([daily_1097, daily_2056, daily_2057])
daily_activity_all = daily_activity_all.dropna(subset=['Whatsapp'])

C:\Users\faruq\AppData\Local\Temp\ipykernel_12308\1821383408.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  daily_activity_all = pd.concat([daily_1097, daily_2056, daily_2057])


In [ ]:
daily_activity_all.rename(columns={
    "Next Action":"kategori_update"
},inplace=True)

In [ ]:
df_daily = status_donor.merge(daily_activity_all, how="left", on="Whatsapp")

In [ ]:
status_donor = status_donor.rename(columns={"Kategori_y":"Kategori"})

In [ ]:
df_daily = df_daily[["Whatsapp","Kategori","kategori_update"]]

In [ ]:
df_daily['kategori_update'].value_counts()

kategori_update
Prioritas 3    2689
Prioirtas 3     592
Prioritas 2     278
Eliminasi       201
Invalid         190
Prioritas 1      88
Name: count, dtype: int64

In [ ]:
def label_status(x):
    status = x['kategori_update']
    kategori = x['Kategori']

    if status in ['Eliminasi', 'Invalid']:
        return 'INVALID'
    else:
        return kategori  # keep original kategori

df_daily['Kategori'] = df_daily.apply(label_status, axis=1)

In [ ]:
df_daily['Kategori'].value_counts()

Kategori
AKTIF      42014
INVALID     1445
LOYAL        142
Name: count, dtype: int64

- update data each crm,. if it's invalid then remove from data each crm

Final output for tracking data

In [ ]:
# def update_dataframe_status(main_df: pd.DataFrame, updated_df: pd.DataFrame, cols_to_check: list) -> pd.DataFrame:
#     common_ids = updated_df.index.intersection(main_df.index)

#     # Use .ne() to detect NaN differences correctly
#     diff_mask = updated_df.loc[common_ids, cols_to_check].ne(main_df.loc[common_ids, cols_to_check]).any(axis=1)

#     rows_to_update = updated_df.loc[common_ids[diff_mask]]
#     new_rows = updated_df.loc[~updated_df.index.isin(main_df.index)]

#     # ✅ Overwrite rows explicitly instead of using .update()
#     main_df.loc[rows_to_update.index, cols_to_check] = rows_to_update[cols_to_check]

#     # Add new rows
#     updated_main_df = pd.concat([main_df, new_rows])
#     updated_main_df = updated_main_df.sort_index()

#     print("✅ Excel updated successfully:")
#     print(f"- {len(new_rows)} new rows added")
#     print(f"- {len(rows_to_update)} rows updated based on {cols_to_check}")

#     return updated_main_df

# FUNCTION FOR CHANGING ONLY CHOSEN COLUMN BASED ON COLUMN CHANGES

In [ ]:
tracking_with_updated_status = update_status_dataframe(
    main_df=tracking_with_crm_updated,
    updated_df=df_daily,
    key_col="Whatsapp",
    cols_to_update=["Kategori"]
)

🆕 Found new columns in updated_df: ['kategori_update']
✅ Update process completed successfully:
- 43602 matched rows updated (['Kategori'])
- 0 new rows added
- 1 new columns added
- Final total rows: 43602


In [ ]:
df_crm['Whatsapp'] = df_crm['Whatsapp'].astype(int)

In [ ]:
# tracking_with_updated_status = tracking_with_updated_status.drop_duplicates(subset=["Whatsapp"], keep="first")
tracking_with_updated_status['Kategori'].value_counts(dropna=False)

Kategori
AKTIF      42014
INVALID     1445
LOYAL        142
NaN            1
Name: count, dtype: int64

In [ ]:
duplicates = tracking_with_updated_status[tracking_with_updated_status['Whatsapp'].duplicated(keep=False)]

if not duplicates.empty:
    print("❌ Duplicate rows found:\n", duplicates)
    raise SystemExit("Stopping cell due to duplicates.")

In [ ]:
tracking_with_updated_status = tracking_with_updated_status[['Whatsapp', 'Donatur', 'Bulan', 'Tahun', 'CRM', 'Januari_Program',
       'Januari_Donasi', 'Januari_Tanggal', 'Januari_Day', 'Januari_Time',
       'Januari_Payment', 'Februari_Program', 'Februari_Donasi',
       'Februari_Tanggal', 'Februari_Day', 'Februari_Time', 'Februari_Payment',
       'Maret_Program', 'Maret_Donasi', 'Maret_Tanggal', 'Maret_Day',
       'Maret_Time', 'Maret_Payment', 'April_Program', 'April_Donasi',
       'April_Tanggal', 'April_Day', 'April_Time', 'April_Payment',
       'Mei_Program', 'Mei_Donasi', 'Mei_Tanggal', 'Mei_Day', 'Mei_Time',
       'Mei_Payment', 'Juni_Program', 'Juni_Donasi', 'Juni_Tanggal',
       'Juni_Day', 'Juni_Time', 'Juni_Payment', 'Juli_Program', 'Juli_Donasi',
       'Juli_Tanggal', 'Juli_Day', 'Juli_Time', 'Juli_Payment',
       'Agustus_Program', 'Agustus_Donasi', 'Agustus_Tanggal', 'Agustus_Day',
       'Agustus_Time', 'Agustus_Payment', 'September_Program',
       'September_Donasi', 'September_Tanggal', 'September_Day',
       'September_Time', 'September_Payment', 'Oktober_Program',
       'Oktober_Donasi', 'Oktober_Tanggal', 'Oktober_Day', 'Oktober_Time',
       'Oktober_Payment', 'November_Program', 'November_Donasi',
       'November_Tanggal', 'November_Day', 'November_Time', 'November_Payment',
       'Desember_Program', 'Desember_Donasi', 'Desember_Tanggal',
       'Desember_Day', 'Desember_Time', 'Desember_Payment', 'Total',
       'Frekuensi', 'klasifikasi_program', 'Preferensi', 'Label_Jam',
       'Rata - rata', 'Day_Mode', 'Date_Category', 'Tahun_Pertama', 'Kategori',
       'Badge', 'Avg Kategori',"Tanggal_terakhir_donasi"]]

In [ ]:
tracking_with_updated_status

,Whatsapp,Donatur,Bulan,Tahun,CRM,Januari_Program,Januari_Donasi,Januari_Tanggal,Januari_Day,Januari_Time,Januari_Payment,Februari_Program,Februari_Donasi,Februari_Tanggal,Februari_Day,Februari_Time,Februari_Payment,Maret_Program,Maret_Donasi,Maret_Tanggal,Maret_Day,Maret_Time,Maret_Payment,April_Program,April_Donasi,April_Tanggal,April_Day,April_Time,April_Payment,Mei_Program,Mei_Donasi,Mei_Tanggal,Mei_Day,Mei_Time,Mei_Payment,Juni_Program,Juni_Donasi,Juni_Tanggal,Juni_Day,Juni_Time,Juni_Payment,Juli_Program,Juli_Donasi,Juli_Tanggal,Juli_Day,Juli_Time,Juli_Payment,Agustus_Program,Agustus_Donasi,Agustus_Tanggal,Agustus_Day,Agustus_Time,Agustus_Payment,September_Program,September_Donasi,September_Tanggal,September_Day,September_Time,September_Payment,Oktober_Program,Oktober_Donasi,Oktober_Tanggal,Oktober_Day,Oktober_Time,Oktober_Payment,November_Program,November_Donasi,November_Tanggal,November_Day,November_Time,November_Payment,Desember_Program,Desember_Donasi,Desember_Tanggal,Desember_Day,Desember_Time,Desember_Payment,Total,Frekuensi,klasifikasi_program,Preferensi,Label_Jam,Rata - rata,Day_Mode,Date_Category,Tahun_Pertama,Kategori,Badge,Avg Kategori,Tanggal_terakhir_donasi
0,628561889392,Muhammad F,February,2025.0,WA_2060,NaN,NaN,NaN,NaN,NaN,NaN,SEDEKAH PALESTINA,100996.0,2025-02-24 05:48:27,Mon,05-06,QRIS,NaN,NaN,NaT,NaN,NaN,NaN,SEDEKAH PALESTINA,100575.0,2025-04-16 05:15:46,Wed,05-06,GOPAY,INFAK PERJUANGAN PALESTINA,201387.0,2025-05-05 05:50:07,Mon,03-04,GOPAY,NaN,NaN,NaT,NaN,NaN,NaN,INFAQ PALESTINA,100376.0,2025-07-23 15:37:53,Wed,15-16,GOPAY,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,300902.0,2025-08-06 05:25:38,Sat,04-05,GOPAY,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,804236.0,5.0,PALESTINA,4.0,05-06,160847.0,Wed,Bukan Gajian,2025.0,LOYAL,SILVER,100.000 - 1000.000,2025-08-26 04:20:28
1,628932345678,Joko Syahputra,February,2025.0,WA_7778,NaN,NaN,NaN,NaN,NaN,NaN,SEDEKAH PALESTINA,78772.0,2025-02-25 04:58:33,Tue,04-05,QRIS,NaN,NaN,NaT,NaN,NaN,NaN,INFAK PALESTINA,201141.0,2025-04-14 00:00:00,Fri,22-23,BCA VA,INFAK PALESTINA,103593.0,2025-05-21 00:00:00,Wed,10-11,BRI VA,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,1054145.0,2025-06-24 06:28:36,Tue,22-23,DANA,INFAQ PALESTINA,566255.0,2025-07-03 08:32:23,Tue,07-08,BRI VA,SEDEKAH PALESTINA,872319.0,2025-08-01 15:36:19,Mon,20-21,QRIS,INFAQ PALESTINA,151292.0,2025-09-01 07:27:43,Mon,02-03,QRIS,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3027517.0,7.0,PALESTINA,4.0,22-23,432502.0,Mon,Bukan Gajian,2025.0,LOYAL,GOLD,100.000 - 1000.000,2025-09-08 02:25:16
2,6282236899125,Orang dari SMAIT Cahaya Bangsa,February,2025.0,WA_2060,NaN,NaN,NaN,NaN,NaN,NaN,SEDEKAH PERJUANGAN PALESTINA,70995.0,2025-02-27 21:37:03,Thu,21-22,GOPAY,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70995.0,1.0,PALESTINA,1.0,21-22,70995.0,Thu,Masa Gajian,2025.0,AKTIF,BRONZE,< 100.000,2025-02-27 21:37:03
3,6282380878760,Hj Asnaina Aliuntu,February,2025.0,WA_2060,NaN,NaN,NaN,NaN,NaN,NaN,SEDEKAH PALESTINA HARI INI,103495.0,2025-02-27 06:22:46,Thu,06-07,BSI,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,100658.0,2025-08-15 02:59:51,Fri,02-03,BRI,BANTU PALESTINA SEKARANG,151649.0,2025-09-08 03:11:53,Mon,02-03,BRI,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,355802.0,3.0,PALESTINA,3.0,02-03,118601.0,Fri,Bukan Gajian,2025.0,AKTIF,SILVER,100.000 - 1000.000,2025-09-10 02:58:41
4,6285753301575,Suminah,February,2025.0,WA_2060,NaN,NaN,NaN,NaN,NaN,NaN,SEDEKAH PALESTINA,100998.0,2025-02-28 06:47:06,Fri,06-07,Mandiri,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN

In [ ]:
tracking_with_updated_status.to_excel("checkpoint_files/tracking_with_updated_category_daily_activity.xlsx")

# Data Waiting that has never been successful before

In [ ]:
df_waiting = data_all[(data_all['is_random'] == True)]
df_waiting = df_waiting[df_waiting['Payment Status'] == 'Waiting']
df_waiting

,No,Invoice ID,Donatur,Sapaan,Nominal,Kode Unik,Total,Whatsapp,Email,Comment,Program,Payment Method,Payment Number,Payment Account,Payment Status,Fundraiser Commission,Fundraiser Name,Date,Day,Time,UTM Source,UTM Medium,ip,Fundraiser Commision,aff_code,zisco,Data Qurban,Data Package-2,Data Zakat Fitrah,Additional Data,UTM Content,UTM Campaign,UTM Term,UTM ID,hour,label_jam,is_random,kategori_nomor
5,6.0,INV-2508040IVSI,Mayang,Bapak,100000.0,446.0,100446.0,6282162426297,NaN,NaN,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,instant,https://m.dana.id/s/m7v7pwbn,DANA,Waiting,1004.0,Teman Baik 1,2025-08-04 00:00:00,Mon,1900-01-01 13:36:00,fb,paid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,13-14,True,Nomor Acak
6,7.0,INV-250804R2C2K,Isep Kamiludin,Bapak,100000.0,528.0,100528.0,6287772391069,NaN,Semoga Allah mengabulkqn doa kita semua..aamiin,BANTU PALESTINA SEKARANG,instant,https://m.dana.id/s/49unkdhk,DANA,Waiting,1005.0,Teman Baik 1,2025-08-04 00:00:00,Mon,1900-01-01 13:36:00,fb,paid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,13-14,True,Nomor Acak
54,55.0,INV-2508049SZY8,A.nabila,Ibu,100000.0,589.0,100589.0,62812812335,NaN,NaN,SEDEKAH PALESTINA,instant,https://flip.id/pwf/transaction/consolidated?r...,QRIS,Waiting,1006.0,Teman Baik 1,2025-08-04 00:00:00,Mon,1900-01-01 10:52:00,fb,paid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,10-11,True,Nomor Acak
56,57.0,INV-250804LQMVG,A.nabila,Ibu,100000.0,209.0,100209.0,62812812335,NaN,NaN,SEDEKAH PALESTINA,transfer,BSI 7772526274,BSI,Waiting,1002.0,Teman Baik 1,2025-08-04 00:00:00,Mon,1900-01-01 10:49:00,fb,paid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,10-11,True,Nomor Acak
72,73.0,INV-250804DZ01S,Didi sutardi.didi1968@gmail.com,Bapak,50000.0,640.0,50640.0,6281324233883,didisutardi0568@gmail.com,Semoga Allah cepat akhiri penderitaan Rakyat d...,BANTU PALESTINA SEKARANG,instant,https://api.midtrans.com/v2/gopay/d1730895-e65...,GOPAY,Waiting,506.0,Teman Baik 1,2025-08-04 00:00:00,Mon,1900-01-01 10:12:00,fb,paid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10,10-11,True,Nomor Acak
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INV-251021VRYNR,NaN,INV-251021VRYNR,Setiono,NaN,NaN,NaN,250024.0,6281287158569,,Ya Allah lindungilah saudara2 kami di Palestin...,SEDEKAH PALESTINA,transfer,BSI 7772526274,BSI,Waiting,NaN,Teman Baik 1,2025-10-21 06:57:21,Tue,1900-01-01 06:57:21,ig,Instagram_Reels,112.78.133.131,2500.0,ecuhf,fredo,[],[],[],"{""Kode Unik"":""24""}",Beben - SurgaMu 2,Beben - SurgaMu 2,120233080358240270,120233080358260270,6,06-07,True,Nomor Acak
INV-251021W19CP,NaN,INV-251021W19CP,NUR FAIZIN,NaN,NaN,NaN,50621.0,6285728851563,,,BANTU PALESTINA SEKARANG,transfer,BNI 3003003257,BNI,Waiting,NaN,Teman Baik 1,2025-10-21 06:50:07,Tue,1900-01-01 06:50:07,ig,Instagram_Reels,103.124.137.132,506.0,3hzvc,fahry,[],[],[],"{""Kode Unik"":""621""}",IDP 1123 UAH Syuhada - Salin,IDP 1123 UAH Syuhada,120234665884310524,120234665883960524,6,06-07,True,Nomor Acak
INV-251021XQEHU,NaN,INV-251021XQEHU,Heni kusumawawati,NaN,NaN,NaN,100509.0,6281398606211,,"Semoga Palestina cepat bebas merdeka, dan Allo...",BANTU PALESTINA SEKARANG,transfer,BRI 115901000647561,BRI,Waiting,NaN,Teman Baik 1,2025-10-21 05:48:01,Tue,1900-01-01 05:48:01,fb,Facebook_Stories,149.108.135.105,1005.0,3hzvc,fahry,[],[],[],"{""Kode Unik"":""509""}",UBN-Bunbun,UBN-Bunbun,120233168788450455,120233168788410455,5,05-06,True,Nomor Acak
INV-251021XYCTC,NaN,INV-251021XYCTC,Frida,NaN,NaN,NaN,50384.0,6287850333830,,,SEDEKAH PALESTINA,instant,https://flip.id/pwf/transaction/consolidated?r...,QRIS,Waiting,NaN,Teman Baik 1,2025-10-21 05:44:35,Tue,1900-01-01 05:44:35,ig,Instagram_Feed,103.18.34.151,504.0,ecuhf,fredo,[],[],[],"{""Kode Unik"":""384""}",IDPOST TB Hook 4,IDPOST TB Hook 4 Kampanye,120232073427280397,120232073427290397,5,05-06,True,Nomor Acak


In [ ]:
agg_waiting = df_waiting.groupby("Whatsapp").agg(
    Donatur=('Donatur','first'),
    Sapaan=('Sapaan','first'),
    Program=('Program', "first"),
    Date=('Date', 'first'),
    Total=("Total", "sum")
).reset_index().sort_values(by='Total', ascending=False)

In [ ]:
agg_waiting['Whatsapp'] = agg_waiting['Whatsapp'].astype(int)

df_merge = pd.merge(agg_waiting,df_tracking, how='left', on='Whatsapp')

NameError: name 'df_tracking' is not defined

In [ ]:
df_merge = df_merge[df_merge['CRM'].isna()]

In [ ]:
only_waiting = df_merge

waiting_final = only_waiting[['Whatsapp','Donatur_x','Sapaan','Program','Date','Total_x']]

In [ ]:
waiting_final['Bulan_Angka'] = pd.to_datetime(waiting_final['Date']).dt.month
waiting_final['Nama_Bulan'] = pd.to_datetime(waiting_final['Date']).dt.strftime('%B')
waiting_final['Nama_Hari'] = pd.to_datetime(waiting_final['Date']).dt.strftime('%A')
waiting_final['Date'] = pd.to_datetime(waiting_final['Date']).dt.date

C:\Users\faruq\AppData\Local\Temp\ipykernel_2100\1233219089.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  waiting_final['Bulan_Angka'] = pd.to_datetime(waiting_final['Date']).dt.month
C:\Users\faruq\AppData\Local\Temp\ipykernel_2100\1233219089.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  waiting_final['Nama_Bulan'] = pd.to_datetime(waiting_final['Date']).dt.strftime('%B')
C:\Users\faruq\AppData\Local\Temp\ipykernel_2100\1233219089.py:3: SettingWithCopyWarning: 
A value is trying to be set on a 

In [ ]:
def check_date(df):
    if df['Nama_Hari'] == 'Friday':
        return 'Jumat'
    elif (df['Date'].day >= 26 or df['Date'].day <= 5):
        return 'Masa Gajian'
    else:
        return 'Bukan Masa Gajian'
    
waiting_final['Status'] = waiting_final.apply(check_date, axis=1)

C:\Users\faruq\AppData\Local\Temp\ipykernel_2100\3927312706.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  waiting_final['Status'] = waiting_final.apply(check_date, axis=1)


In [ ]:
waiting_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10358 entries, 0 to 12571
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Whatsapp     10358 non-null  int64  
 1   Donatur_x    10351 non-null  object 
 2   Sapaan       925 non-null    object 
 3   Program      10358 non-null  object 
 4   Date         10358 non-null  object 
 5   Total_x      10358 non-null  float64
 6   Bulan_Angka  10358 non-null  int32  
 7   Nama_Bulan   10358 non-null  object 
 8   Nama_Hari    10358 non-null  object 
 9   Status       10358 non-null  object 
dtypes: float64(1), int32(1), int64(1), object(7)
memory usage: 849.7+ KB


In [ ]:
waiting_final.to_excel('Data_waiting.xlsx')

In [ ]:
# df_database = pd.read_excel('Tracking_temanbaik.xlsx')
df_database = df_tracking.copy()

In [ ]:
df_database['Bulan'].value_counts(dropna=False)

Bulan
August       10410
July          8542
June          6836
September     6266
May           3657
April         3351
October       2277
March          921
February         8
January          0
November         0
December         0
Name: count, dtype: int64

In [ ]:
df_database[df_database['Label_Jam'].isna()]

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Whatsapp,Donatur,Bulan,Tahun,CRM,Januari_Program,Januari_Donasi,Januari_Tanggal,Januari_Day,Januari_Time,Januari_Payment,Februari_Program,Februari_Donasi,Februari_Tanggal,Februari_Day,Februari_Time,Februari_Payment,Maret_Program,Maret_Donasi,Maret_Tanggal,Maret_Day,Maret_Time,Maret_Payment,April_Program,April_Donasi,April_Tanggal,April_Day,April_Time,April_Payment,Mei_Program,Mei_Donasi,Mei_Tanggal,Mei_Day,Mei_Time,Mei_Payment,Juni_Program,Juni_Donasi,Juni_Tanggal,Juni_Day,Juni_Time,Juni_Payment,Juli_Program,Juli_Donasi,Juli_Tanggal,Juli_Day,Juli_Time,Juli_Payment,Agustus_Program,Agustus_Donasi,Agustus_Tanggal,Agustus_Day,Agustus_Time,Agustus_Payment,September_Program,September_Donasi,September_Tanggal,September_Day,September_Time,September_Payment,Oktober_Program,Oktober_Donasi,Oktober_Tanggal,Oktober_Day,Oktober_Time,Oktober_Payment,November_Program,November_Donasi,November_Tanggal,November_Day,November_Time,November_Payment,Desember_Program,Desember_Donasi,Desember_Tanggal,Desember_Day,Desember_Time,Desember_Payment,Total,Frekuensi,klasifikasi_program,Preferensi,Label_Jam,Rata - rata,Day_Mode,Date_Category,Tahun_Pertama,Kategori,Badge,Avg Kategori
4360,4359.0,4334.0,4333.0,4326.0,4305.0,628529434217,Elis Maryati,July,2025,WA_7778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,INFAQ UNTUK PALESTINA,50000.0,2025-07-23,Wed,NaN,BRI,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50000.0,1,PALESTINA,1,NaN,50000,Wed,Bukan Gajian,2025,AKTIF,BRONZE,< 100.000
4728,4726.0,4698.0,4697.0,4690.0,4667.0,628811972538,Nisa Mauldy Julianti (E-Book),June,2025,WA_2062,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,SEDEKAH JUMAT PALESTINA,40000.0,2025-06-20 00:00:00,Fri,NaN,BSI,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40000.0,1,PALESTINA,1,NaN,40000,Fri,Bukan Gajian,2025,AKTIF,BRONZE,< 100.000
7991,7989.0,7942.0,7941.0,7927.0,7895.0,6281253694272,Puji Widyastanti,June,2025,WA_7778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,202899.0,2025-06-02 03:51:54,Fri,NaN,BRI VA,SEDEKAH JUM'AT PALESTINA,100000.0,2025-07-04,Fri,NaN,BRI,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,302899.0,2,PALESTINA,2,NaN,151450,Fri,Masa Gajian,2025,AKTIF,SILVER,100.000 - 1000.000
22916,22901.0,22791.0,22784.0,22751.0,22672.0,6282241631636,Syahrul Nasution,June,2025,WA_7778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,INFAQ PALESTINA,50000.0,2025-06-17 00:00:00,Tue,NaN,BSI,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50000.0,1,PALESTINA,1,NaN,50000,Tue,Bukan Gajian,2025,AKTIF,BRONZE,< 100.000
24752,24735.0,24618.0,24610.0,24576.0,24492.0,6282293001020,Gafar Ahmad,June,2025,WA_7778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,INFAK PALESTINA: BENTUK PEDULI KITA KEPADA SAU...,300000.0,2025-06-01 17:44:26,Fri,NaN,BSI,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,300000.0,1,PALESTINA,1,NaN,300000,Fri,Masa Gajian,2025,AKTIF,SILVER,100.000 - 1000.000
25591,25574.0,25448.0,25440.0,25405.0,25321.0,6282332999495,Hasan Ashari,June,2025,WA_7778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,N